[**Dynamically defining a struct with default values**](https://discourse.julialang.org/t/dynamically-defining-a-struct-with-default-values/106855)

I would like to be able to define a struct with some default values based on an input Dict. I would like to define a struct like below:

```julia
d = Dict("A"=>1, "B"=>2, "C"=>"x", "D"=>[1,2])
@kwdef struct MyStruct3
	A::Int64 = 1
	B::Int64 = 2
	C::String = "x"
    D::Vector{Int64} = [1,2]
end
```

In [1]:
d = Dict("A"=>1, "B"=>2, "C"=>"x", "D"=>[1,2])

Dict{String, Any} with 4 entries:
  "B" => 2
  "A" => 1
  "C" => "x"
  "D" => [1, 2]

In [6]:
macro make_struct(name, dict)
	symbols=[:($(Symbol(k))::$(typeof(v))) for (k, v) in eval(dict)]
	:(@kwdef struct $(esc(name))
    	$(map(esc,symbols)...)
    end)
end
@make_struct Test d
Test |> typeof |> display

DataType

In [8]:
# But am not sure how to add default values to that, when I tried this:
macro make_struct(name, dict)
	symbols=[:($(Symbol(k))::$(typeof(v))=$(v)) for (k, v) in eval(dict)]
	:(@kwdef struct $(esc(name))
    	$(map(esc,symbols)...)
    end)
end
@make_struct Test2 d

ErrorException: syntax: "B::Int64 = 2" inside type definition is reserved around util.jl:609

In [9]:
struct Hu
    A::Int64 = 1
end

ErrorException: syntax: "A::Int64 = 1" inside type definition is reserved around e:\JuliaProjects\Training.jl\struct\struct.ipynb:1

In [10]:
let name = :Ha
    @eval @kwdef struct $(esc(name)) end
end

ErrorException: syntax: invalid type signature around util.jl:609

In [17]:
using General.Aux

In [19]:
function def_dyn_struct(name::Symbol, spec::Dict)
    fields = [:($(Symbol(k))::$(typeof(v)) = $(v)) for (k, v) in spec]
    fields |> display
    @eval @kwdef struct $(name)
        $(fields...)
    end
end
def_dyn_struct(:Test3, d)
t3 = Test3()
@logt t3 t3.D

4-element Vector{Expr}:
 :(B::Int64 = 2)
 :(A::Int64 = 1)
 :(C::String = "x")
 :(D::Vector{Int64} = [1, 2])


t3, Test3
  = Test3(2, 1, "x", [1, 2])

t3.D, Vector{Int64}
  = [1, 2]


Glad you have a solution. FWIW, if you ever wanted full dynamism, NamedTuples are the way to get it. Something like

In [20]:
struct Flexible{NT}
    fields::NT
end
mk_struct_constructor(dict::Dict) = mk_struct_constructor(NamedTuple(dict))
Base.getproperty(f::Flexible, field) = getproperty(f.fields, field)
function mk_struct_constructor(nt::NamedTuple)
    constructor(; kwargs...) = Flexible(merge(nt, NamedTuple(kwargs)))
end

mk_struct_constructor (generic function with 2 methods)

In [30]:
MyStruct = mk_struct_constructor(Dict(:A=>2))
ms1 = MyStruct()
@logt MyStruct ms1


MyStruct, var"#constructor#46"{var"#constructor#45#47"{@NamedTuple{A::Int64}}}
  = constructor

ms1, Flexible{@NamedTuple{A::Int64}}
  = Flexible{@NamedTuple{A::Int64}}((A = 2,))


In [31]:
ms2 = MyStruct()

Flexible{@NamedTuple{A::Int64}}((A = 2,))

In [32]:
ms3 = MyStruct(A=5)

Flexible{@NamedTuple{A::Int64}}((A = 5,))

This won’t face the world-age issues you’d have with the previous solution, and it’s AFAIK just as efficient as a normal struct.

这不会像之前的解决方案那样面临世界级问题，而且据我所知，其效率与普通的结构相同。